# PETROV CLASSIFICATION OF SPACE-TIME IN ENTANGLED RELATIVITY FOR BOTH ELECTRICALLY AND MAGNETICALLY BLACK HOLES IN SLOW ROTATION

This notebook's main objective is to characterize spacetime in the case of a slowly rotating black hole within the framework of entangled relativity (ER). Here, we use the Petrov classification criteria. As external solutions of isolated bodies characterized by their mass and angular momentum, we expect to find a type D black hole. 

We will start by introducing the ER metric, then we will calculate the Weyl tensor at first order in 'a' (the black hole's angular momentum), and finally, we will determine the NP-Weyl scalars to define the type of spacetime.

The complexity of some calculations in this notebook prompted us to perform intermediate first order Taylor expansions to maintain tractability of the different operations using the Sagemath language. This does not affect the validity of our results since they are part of a first-order analysis in 'a'.

In [ ]:
version()

In [ ]:
%display latex

'SageMath version used is 10.1, Release Date: 2023-08-20'

In [ ]:
from sage.manifolds.operators import dalembertian
from sage.manifolds.operators import grad
from sage.tensor.modules.tensor_with_indices import TensorWithIndices
from IPython.display import clear_output
import time
import pickle
comput_time0 = time.perf_counter()

Let's write a function to replace all the different expressions and perform the taylor expansions 

In [ ]:
def slow_rot(arg):
    subs_funcs = [(Z, E), (U, A), (V, B), (X, C), (Y, D)]
    final_subs = {Q: H}
    
    if hasattr(arg, 'expr'):
        arg = arg.expr()
        
    if hasattr(arg, 'apply_map')*hasattr(arg, 'display'):
        arg.apply_map(lambda f: f.taylor(a, 0, 1))
        for i, (old_func, new_func) in enumerate(subs_funcs):
            arg.apply_map(lambda f: (f.substitute_function(old_func, new_func).taylor(a, 0, 1) if i == 0 \
                                           else f.substitute_function(old_func, new_func)))                        
        arg.apply_map(lambda f: f.subs(final_subs).factor())
    else:
        arg = arg.taylor(a, 0, 1)
        for i, (old_func, new_func) in enumerate(subs_funcs):
            arg = (arg.substitute_function(old_func, new_func).taylor(a, 0, 1) if i == 0 \
                                           else arg.substitute_function(old_func, new_func)) 
        arg = arg.subs(final_subs).factor()
    
    return arg

# I. Definition of the metric in ER

In [ ]:
M = Manifold(4, 'M', structure='Lorentzian')
print(M)

In [ ]:
X.<t,r,th,ph> = M.chart(r't r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi')
X

In [ ]:
g = M.metric()
r_m, r_p, a, Q = var('r_m r_p a Q')

In [ ]:
U = function('U')
V = function('V')
X = function('X')
Y = function('Y')
Z = function('Z')
O = function('O')
g[0,0] = U(r)
g[0,3] = Z(r,th) 
g[1,1] = V(r)
g[2,2] = X(r)
g[3,3] = Y(r,th)
g[3,0] = Z(r,th) 

In [ ]:
elec_case = false # Here we are considering the magnetic case if elec_case=False

In [ ]:
Psi(r) = (1-r_p/r)
Zeta(r) = (1-r_m/r)

if elec_case:
    A(r) = -Psi(r)*Zeta(r)**(15/13)
    B(r) = 1/(Psi(r)*Zeta(r)**(7/13))
    C(r) = r**2*Zeta(r)**(6/13)
    D(r) = (r*sin(th))**2*Zeta(r)**(6/13)
    f(r)=(r**2*(1+(1/(2*(3)**(1/2)))**2)**2)*Zeta(r)**(2*(1/(2*(3)**(1/2)))**2/(1+(1/(2*(3)**(1/2)))**2))/(((1-(1/(2*(3)**(1/2)))**2)*(1-3*(1/(2*(3)**(1/2)))**2)*r_m**2))-Zeta(r)**((1-(1/(2*(3)**(1/2)))**2)/(1+(1/(2*(3)**(1/2)))**2))*(1+(1+(1/(2*(3)**(1/2)))**2)**2*r**2/((1-(1/(2*(3)**(1/2)))**2)*(1-3*(1/(2*(3)**(1/2)))**2)*r_m**2)+(1+(1/(2*(3)**(1/2)))**2)*r/((1-(1/(2*(3)**(1/2)))**2)*r_m)-r_p/r)
    E(r,th)=-a*sin(th)**2*f(r)*Zeta(r)**(4/13)

else:
    A(r) = -Psi(r)*Zeta(r)**(7/13)
    B(r) = 1/(Psi(r)*Zeta(r)**(15/13))
    C(r) = r**2*Zeta(r)**(-2/13)
    D(r) = (r*sin(th))**2*Zeta(r)**(-2/13)
    f(r)=(r**2*(1+(1/(2*(3)**(1/2)))**2)**2)*Zeta(r)**(2*(1/(2*(3)**(1/2)))**2/(1+(1/(2*(3)**(1/2)))**2))/(((1-(1/(2*(3)**(1/2)))**2)*(1-3*(1/(2*(3)**(1/2)))**2)*r_m**2))-Zeta(r)**((1-(1/(2*(3)**(1/2)))**2)/(1+(1/(2*(3)**(1/2)))**2))*(1+(1+(1/(2*(3)**(1/2)))**2)**2*r**2/((1-(1/(2*(3)**(1/2)))**2)*(1-3*(1/(2*(3)**(1/2)))**2)*r_m**2)+(1+(1/(2*(3)**(1/2)))**2)*r/((1-(1/(2*(3)**(1/2)))**2)*r_m)-r_p/r)
    E(r,th)=-a*sin(th)**2*f(r)*Zeta(r)**(-4/13)

Let's now define the mass and the charge expressions

In [ ]:
Ma = (11/26)*r_m+r_p/2 
H=(r_m*r_p/(1+(1/(2*(3)**(1/2)))**2))**(1/2)

In [ ]:
h = g.copy()
h.set_name('h')
slow_rot(h).display()

# II. Computation of the Weyl tensor

\begin{eqnarray}
C_{abcd} &=& R_{abcd} - \frac{1}{2}(g_{a[c}R_{d]b} - g_{b[c}R_{d]a}) + \frac{1}{6}R g_{a[c}g_{d]b}\\
&=& R_{abcd} - \frac{1}{2}\left(R_{ac}.g_{bd}-R_{ad}.g_{bc}+R_{bd}.g_{ac}-R_{bc}.g_{ad}\right)+\frac{1}{6}\left(g_{ac}g_{bd}-g_{ad}g_{bc}\right)R
\end{eqnarray}

We will not use the .weyl() property of SageMath here due to the complexity of the calculations and the need to progressively simplify the expressions through a Taylor expansion to first order in 'a'.

In [ ]:
g.display()

## II.1. The Riemann and Ricci tensors

In [ ]:
ER_riem = g.riemann()
slow_rot(ER_riem)
ER_riem = ER_riem.down(g,0)
slow_rot(ER_riem)
ER_riem = TensorWithIndices(ER_riem, '_abcd')

In [ ]:
ER_ricc = g.ricci()
slow_rot(ER_ricc)

In [ ]:
gu = g.up(g)
slow_rot(gu)

In [ ]:
ER_rsc = gu['^ab']*ER_ricc['_ab']
show(LatexExpr(r'R = '), slow_rot(ER_rsc), LatexExpr(r'+ \mathcal{O}(a^{2})'))

## II.2. Antisymmetrization

1. Let's compute $Rg_{a[c}g_{d]b}$

In [ ]:
g_prod = ER_rsc*g*g
slow_rot(g_prod)
gp_anti = g_prod['_acbd']-g_prod['_adbc']

antisymmetrize$\lbrace1,2\rbrace(g_{ac}g_{bd})\rightarrow \frac{1}{2}\left(g_{ac}g_{bd}-g_{ad}g_{bc}\right)=\frac{1}{2}g_{a[c}g_{b]d}$

2. Let's now compute $g_{a[c}R_{d]b} - g_{b[c}R_{d]a}$

In [ ]:
gR = g*ER_ricc
slow_rot(gR)
gR_prod = gR['_acbd']-gR['_adbc']+gR['_bdac']-gR['_bcad']

# III. The Weyl tensor

In [ ]:
ER_Weyl = ER_riem-(1/2)*gR_prod+(1/6)*gp_anti

for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                latex_str = r'C_{{{}{}{}{}}} = '.format(i, j, k, l)  
                if [i, j, k, l] in [[0, 1, 0, 1],[0, 1, 1, 0],[0, 1, 1, 3],[0, 1, 2, 3],[0, 1, 3, 2]]:
                    show(LatexExpr(latex_str), ER_Weyl[i, j, k, l].expr().canonicalize_radical().factor().factor()) 


# IV. NP-Weyl scalars

Let's import the tetrad expression in ER derived in the notebook entitled "ER_SR_BH_Tetrad"

In [ ]:
with open('SR_electric_tetrad_.pkl' if elec_case else 'SR_magnetic_tetrad_.pkl', 'rb') as file:
    expressions = pickle.load(file)
    
L = expressions['L_expression']
N = expressions['N_expression']
Mr = expressions['M_expression']
Mbar = expressions['Mbar_expression']

show(LatexExpr(r'l^{\mu} = '),L[:])
show(LatexExpr(r'n^{\mu} = '),N[:])
show(LatexExpr(r'm^{\mu} = '),Mr[:])
show(LatexExpr(r'\bar{m}^{\mu} = '), Mbar[:])

We need to convert the tetrad into a M manifold indexed vector field and align it onto the same tetrad

In [ ]:
tetrad_L, tetrad_N, tetrad_Mr,tetrad_Mbar = M.vector_field(name='tetrad_L'), M.vector_field(name='tetrad_N'), \
                                            M.vector_field(name='tetrad_Mr'), M.vector_field(name='tetrad_Mbar')
for i in range(4):
    tetrad_L[i] = L[i].expr()
    tetrad_N[i] = N[i].expr()
    tetrad_Mr[i] = Mr[i].expr()
    tetrad_Mbar[i] = Mbar[i].expr()

In [ ]:
components = [[[[ER_Weyl[i,j,k,l] for i in range(4)] for j in range(4)] for k in range(4)] for l in range(4)]
Weyl_tensor = M.tensor_field(0, 4, name='Weyl_tensor')
Weyl_tensor[:, :, :, :] = components

## IV.1. Scalar 0

In [ ]:
print('1st contraction', Weyl_tensor.tensor_type())
Psi0 = Weyl_tensor.contract(0, tetrad_L, 0)
Psi0.apply_map(lambda u: u.taylor(a,0,1))
print('2nd contraction', Psi0.tensor_type())
Psi0 = Psi0.contract(0, tetrad_Mr, 0)
Psi0.apply_map(lambda u: u.taylor(a,0,1))
print('3rd contraction', Psi0.tensor_type())
Psi0 = Psi0.contract(0, tetrad_L, 0)
Psi0.apply_map(lambda u: u.taylor(a,0,1))
print('4th contraction', Psi0.tensor_type())
Psi0 = Psi0.contract(0, tetrad_Mr, 0)
Psi0 = Psi0.expr().taylor(a,0,1)
latex_str = r'\Psi_{0} = W_{\alpha\beta\gamma\delta}l^{\alpha}m^{\beta}l^{\gamma}m^{\delta} = ' 
show(LatexExpr(latex_str), Psi0)
clear_output(wait=True)

## IV.2. Scalar I

In [ ]:
print('1st contraction', Weyl_tensor.tensor_type())
Psi1 = Weyl_tensor.contract(0, tetrad_L, 0)
Psi1.apply_map(lambda u: u.taylor(a,0,1))
print('2nd contraction', Psi1.tensor_type())
Psi1 = Psi1.contract(0, tetrad_N, 0)
Psi1.apply_map(lambda u: u.taylor(a,0,1))
print('3rd contraction', Psi1.tensor_type())
Psi1 = Psi1.contract(0, tetrad_L, 0)
Psi1.apply_map(lambda u: u.taylor(a,0,1))
print('4th contraction', Psi1.tensor_type())
Psi1 = Psi1.contract(0, tetrad_Mr, 0)
Psi1 = Psi1.expr().taylor(a,0,1)
latex_str = r'\Psi_{1} = W_{\alpha\beta\gamma\delta}l^{\alpha}n^{\beta}l^{\gamma}m^{\delta} = ' 
show(LatexExpr(latex_str), Psi1)
clear_output(wait=True)

## IV.3. Scalar II

In [ ]:
print('1st contraction', Weyl_tensor.tensor_type())
Psi2 = Weyl_tensor.contract(0, tetrad_L, 0)
Psi2.apply_map(lambda u: u.taylor(a,0,1))
print('2nd contraction', Psi2.tensor_type())
Psi2 = Psi2.contract(0, tetrad_Mr, 0)
Psi2.apply_map(lambda u: u.taylor(a,0,1))
print('3rd contraction', Psi2.tensor_type())
Psi2 = Psi2.contract(0, tetrad_Mbar, 0)
Psi2.apply_map(lambda u: u.taylor(a,0,1))
print('4th contraction', Psi2.tensor_type())
Psi2 = Psi2.contract(0, tetrad_N, 0)
Psi2 = Psi2.expr().taylor(a,0,1)
latex_str = r'\Psi_{2} = W_{\alpha\beta\gamma\delta}l^{\alpha}m^{\beta}\bar{m}^{\gamma}n^{\delta} = ' 
show(LatexExpr(latex_str), Psi2)
clear_output(wait=True)

## IV.4. Scalar III

In [ ]:
print('1st contraction', Weyl_tensor.tensor_type())
Psi3 = Weyl_tensor.contract(0, tetrad_L, 0)
Psi3.apply_map(lambda u: u.taylor(a,0,1))
print('2nd contraction', Psi3.tensor_type())
Psi3 = Psi3.contract(0, tetrad_N, 0)
Psi3.apply_map(lambda u: u.taylor(a,0,1))
print('3rd contraction', Psi3.tensor_type())
Psi3 = Psi3.contract(0, tetrad_Mbar, 0)
Psi3.apply_map(lambda u: u.taylor(a,0,1))
print('4th contraction', Psi3.tensor_type())
Psi3 = Psi3.contract(0, tetrad_N, 0)
Psi3 = Psi3.expr().taylor(a,0,1)
latex_str = r'\Psi_{3} = W_{\alpha\beta\gamma\delta}l^{\alpha}n^{\beta}\bar{m}^{\gamma}n^{\delta} = ' 
show(LatexExpr(latex_str), Psi3)
clear_output(wait=True)

## IV.5. Scalar IV

In [ ]:
print('1st contraction', Weyl_tensor.tensor_type())
Psi4 = Weyl_tensor.contract(0, tetrad_N, 0)
Psi4.apply_map(lambda u: u.taylor(a,0,1))
print('2nd contraction', Psi4.tensor_type())
Psi4 = Psi4.contract(0, tetrad_Mbar, 0)
Psi4.apply_map(lambda u: u.taylor(a,0,1))
print('3rd contraction', Psi4.tensor_type())
Psi4 = Psi4.contract(0, tetrad_N, 0)
Psi4.apply_map(lambda u: u.taylor(a,0,1))
print('4th contraction', Psi4.tensor_type())
Psi4 = Psi4.contract(0, tetrad_Mbar, 0)
Psi4 = Psi4.expr().taylor(a,0,1)
latex_str = r'\Psi_{4} = W_{\alpha\beta\gamma\delta}n^{\alpha}\bar{m}^{\beta}n^{\gamma}\bar{m}^{\delta} = ' 
show(LatexExpr(latex_str), Psi4)
clear_output(wait=True)

# V. Characteristics of $\Psi_{1}$, $\Psi_{2}$ and $\Psi_{3}$

## V.I. Near vacuum limit of $\Psi_{1}$, $\Psi_{3}$

In the case of small $r_{-}$ we can write:

In [ ]:
show(LatexExpr(r'\Psi_{1} = '),Psi1.taylor(r_m,0,1),LatexExpr(r' + \mathcal{O}(r_m^2)'))
show(LatexExpr(r'\Psi_{3} = '),Psi3.taylor(r_m,0,1),LatexExpr(r' + \mathcal{O}(r_m^2)'))

By testing for arbitrary values of $r_{m}$ and $r_{p}$ (Sage is unable to determine whether such an expression is imaginary or real in the general case), we observe that $\Psi_{1}$ and $\Psi_{3}$ are pure imaginary numbers but not zero! However, in the nearly empty limit these quantites do reduce to zero.

## V.I. Near vacuum limit of $\Psi_{2}$

Let's check that in the near-vaccum case ($r_{m}=0$), the NP-Weyl scalar $\Psi_{2}$ has the same expression that in the case of the Kerr truncated solution at first order in $a$:

$\Psi_{2}^{KERR}=-\frac{{\left(-2 i \, Q^{2} + 3 i \, M r\right)} a \cos\left({\theta}\right)}{r^{5}} + \frac{Q^{2} - M r}{r^{4}}$


Let's start from the Kerr truncated solution and replacing $M$ and $Q$ by their expressions, we'll show that we end up with: 

$\lim_{r_m \to 0}\Psi_{2} = \Psi_{2}^{KERR}$

In [ ]:
show(LatexExpr('M = '), Ma)
show(LatexExpr('Q = '), H)

In [ ]:
Im = -(-2*H**2+3*Ma*r)*a*cos(th)/(r**5)
Re = (H**2-Ma*r)/(r**4)
show(LatexExpr(r'\Psi_{2}^{KERR} = '), (Im*I+Re).subs(r_m=0))

Finally, we verify

In [ ]:
show(LatexExpr(r'\Psi_{2} = '),Psi2.taylor(r_m,0,1),LatexExpr(r' + \mathcal{O}(r_m^2)'))

In [ ]:
show(LatexExpr(r'\lim_{r_m \to 0}\Psi_{2} = '),Psi2.subs(r_m=0),LatexExpr(r' = \Psi_{2}^{KERR}'))

So in vacuum, this corresponds to the truncated Kerr solution for slow rotations.

In [ ]:
clear_output(wait=True)

# In conclusion, we cannot say that we have a type D solution as the NP-Weyl scalars values escape the Petrov classification for both the electric and magnetic cases, but in the nearly empty limit, we recover the Petrov classification of the Kerr solution.

## /!\ Due to the memory space required to perform these calculations and to facilitate reopening the notebook, we strongly encourage you to delete all outputs (Kernel -> Restart & Clear Output) before closing /!\